In [4]:
!pip install opencv-python numpy tensorflow scikit-learn matplotlib lxml

In [8]:
pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import cv2
import xml.etree.ElementTree as ET

def load_data(images_path, annotations_path):
    images = []
    annotations = []
    image_filenames_with_annotations = []

    # Step 1: Gather all image filenames that have corresponding annotations
    for xml_file in os.listdir(annotations_path):
        if xml_file.endswith('.xml'):
            tree = ET.parse(os.path.join(annotations_path, xml_file))
            root = tree.getroot()
            image_file = root.find('filename').text
            image_filenames_with_annotations.append(image_file)

    # Step 2: Load only the images that have corresponding annotations
    for image_file in os.listdir(images_path):
        if image_file in image_filenames_with_annotations:
            image_path = os.path.join(images_path, image_file)
            image = cv2.imread(image_path)
            if image is not None:
                images.append(image)

    # Step 3: Load corresponding annotations
    for xml_file in os.listdir(annotations_path):
        if xml_file.endswith('.xml'):
            tree = ET.parse(os.path.join(annotations_path, xml_file))
            root = tree.getroot()
            image_file = root.find('filename').text
            if image_file in image_filenames_with_annotations:
                img_annotations = []
                for obj in root.findall('object'):
                    class_id = obj.find('name').text
                    bndbox = obj.find('bndbox')
                    x_min = int(bndbox.find('xmin').text)
                    y_min = int(bndbox.find('ymin').text)
                    x_max = int(bndbox.find('xmax').text)
                    y_max = int(bndbox.find('ymax').text)
                    img_annotations.append((class_id, (x_min, y_min, x_max, y_max)))
                annotations.append((image_file, img_annotations))

    return images, annotations

# Paths to dataset
train_images_path = r"C:\Users\Sai Madhu\OneDrive\Desktop\code\road-damage-detection\data_ambi\RDD2022_India\India\train\images"
train_annotations_path = r"C:\Users\Sai Madhu\OneDrive\Desktop\code\road-damage-detection\data_ambi\RDD2022_India\India\train\annotations\xmls"

# Load data
train_images, train_annotations = load_data(train_images_path, train_annotations_path)

# Step 4: Print the number of images and annotations after filtering
print(f"Number of images: {len(train_images)}, Number of annotations: {len(train_annotations)}")


Number of images: 7706, Number of annotations: 7706


python -c "import sys; print(sys.executable)"


In [10]:
print(f"Number of images: {len(train_images)}, Number of annotations: {len(train_annotations)}")

Number of images: 7706, Number of annotations: 7706


In [11]:
import numpy as np

# Resize all images to a fixed size (224x224 for example) and normalize annotations
def preprocess_data(images, annotations, target_size=(224, 224)):
    processed_images = []
    processed_annotations = []

    for img, ann in zip(images, annotations):
        # Resize image
        img_resized = cv2.resize(img, target_size)
        processed_images.append(img_resized)
        
        # Normalize annotations (bounding boxes)
        img_annotations = []
        for class_id, (x_min, y_min, x_max, y_max) in ann[1]:
            img_height, img_width, _ = img.shape
            x_min_norm = x_min / img_width
            y_min_norm = y_min / img_height
            x_max_norm = x_max / img_width
            y_max_norm = y_max / img_height
            img_annotations.append((class_id, (x_min_norm, y_min_norm, x_max_norm, y_max_norm)))
        processed_annotations.append(img_annotations)

    return np.array(processed_images), processed_annotations

# Preprocess the loaded data
X_train, y_train = preprocess_data(train_images, train_annotations)

In [12]:
print(f"X_train: {len(X_train)}, y_train: {len(y_train)}")

X_train: 7706, y_train: 7706


In [13]:
print(f"X_train: {len(X_train)}, y_train: {len(y_train)}")

X_train: 7706, y_train: 7706


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")

Training samples: 6164, Validation samples: 1542


In [15]:
print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")

Training samples: 6164, Validation samples: 1542


In [17]:
pip install tensorflow

  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached grpcio-1.67.1-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.6.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached ml_dtypes-0.4.1-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using 

In [18]:
# Step 1: Import Required Libraries

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import Sequence

In [25]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

class AugmentedObjectDetectionGenerator(Sequence):
    def __init__(self, images, annotations, batch_size=16, input_size=(224, 224)):
        self.images = images
        self.annotations = annotations
        self.batch_size = batch_size
        self.input_size = input_size
        self.datagen = ImageDataGenerator(
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            fill_mode='nearest'
        )

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, index):
        batch_images = self.images[index * self.batch_size:(index + 1) * self.batch_size]
        batch_annotations = self.annotations[index * self.batch_size:(index + 1) * self.batch_size]
        return self.__data_generation(batch_images, batch_annotations)

    def __data_generation(self, batch_images, batch_annotations):
        X = np.zeros((len(batch_images), *self.input_size, 3))
        y = np.zeros((len(batch_images), 4))

        for i, (img, ann) in enumerate(zip(batch_images, batch_annotations)):
            img = img / 255.0  # Normalize the image

            # Apply data augmentation to the image
            img_augmented = self.datagen.random_transform(img)
            X[i,] = img_augmented

            if ann:
                class_id, (x_min, y_min, x_max, y_max) = ann[0]
                y[i,] = [x_min, y_min, x_max, y_max]

        return X, y


In [27]:
import tensorflow as tf
print(tf.__version__)
print("Is TensorFlow using GPU?:", tf.config.list_physical_devices('GPU'))

2.18.0
Is TensorFlow using GPU?: []


In [32]:
import tensorflow as tf

# Check TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

# Check if GPU is available
print("Is TensorFlow using GPU?:", tf.config.list_physical_devices('GPU'))

# Test a simple operation on the GPU
if tf.config.list_physical_devices('GPU'):
    with tf.device('/GPU:0'):
        a = tf.random.normal([1000, 1000])
        b = tf.matmul(a, a)
        print(b)


TensorFlow version: 2.18.0
Is TensorFlow using GPU?: []


In [33]:
nvcc --version


NameError: name 'nvcc' is not defined

In [3]:
python --version


NameError: name 'python' is not defined

In [2]:
pip install tensorflow==2.11.0

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.11.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==2.11.0


In [26]:
# Step 1: Import Required Libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Step 2: Check GPU Availability
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    try:
        # Set memory growth to avoid memory allocation issues
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print(f"GPU is available and enabled: {physical_devices[0]}")
    except RuntimeError as e:
        print(f"Error enabling GPU: {e}")
else:
    print("No GPU found, using CPU.")

# Step 3: Define the RCNN Model
def create_rcnn_model(input_shape):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)  # Increased the number of neurons
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(4, activation='sigmoid')(x)

    model = models.Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

# Step 4: Create the Model
model = create_rcnn_model((224, 224, 3))
model.summary()


No GPU found, using CPU.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape          ┃      Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 224, 224, 3)   │            0 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block1_conv1 (Conv2D)         │ (None, 224, 224, 64)  │        1,792 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block1_conv2 (Conv2D)         │ (None, 224, 224, 64)  │       36,928 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block1_pool (MaxPooling2D)    │ (None, 112, 112, 64)  │            0 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block2_conv1 (Conv2D)         │ (None, 112, 112, 128) │       73,856 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block2_conv2 (Conv2D)         │ (None, 112, 112, 128) │      147,584 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block2_pool (MaxPooling2D)    │ (None, 56, 56, 128)   │            0 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block3_conv1 (Conv2D)         │ (None, 56, 56, 256)   │      295,168 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block3_conv2 (Conv2D)         │ (None, 56, 56, 256)   │      590,080 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block3_conv3 (Conv2D)         │ (None, 56, 56, 256)   │      590,080 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block3_pool (MaxPooling2D)    │ (None, 28, 28, 256)   │            0 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block4_conv1 (Conv2D)         │ (None, 28, 28, 512)   │    1,180,160 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block4_conv2 (Conv2D)         │ (None, 28, 28, 512)   │    2,359,808 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block4_conv3 (Conv2D)         │ (None, 28, 28, 512)   │    2,359,808 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block4_pool (MaxPooling2D)    │ (None, 14, 14, 512)   │            0 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block5_conv1 (Conv2D)         │ (None, 14, 14, 512)   │    2,359,808 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block5_conv2 (Conv2D)         │ (None, 14, 14, 512)   │    2,359,808 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block5_conv3 (Conv2D)         │ (None, 14, 14, 512)   │    2,359,808 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ block5_pool (MaxPooling2D)    │ (None, 7, 7, 512)     │            0 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ flatten_1 (Flatten)           │ (None, 25088)         │            0 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ dense_2 (Dense)               │ (None, 512)           │   12,845,568 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ dropout_1 (Dropout)           │ (None, 512)           │            0 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ dense_3 (Dense)               │ (None, 4)             │        2,052 │
└───────────────────────────────┴───────────────────────┴──────────────┘

 Total params: 27,562,308 (105.14 MB)

 Trainable params: 12,847,620 (49.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [21]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [22]:
# Define batch size
batch_size = 16

# Create the custom data generators
train_generator = AugmentedObjectDetectionGenerator(X_train, y_train, batch_size=batch_size)
val_generator = AugmentedObjectDetectionGenerator(X_val, y_val, batch_size=batch_size)


In [23]:
# Train the model
model.fit(train_generator, validation_data=val_generator, epochs=50, callbacks=[early_stopping])


C:\Users\Sai Madhu\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
 83/386 ━━━━━━━━━━━━━━━━━━━━ 5:46 1s/step - loss: 0.2545 - mae: 0.3417

KeyboardInterrupt: 